## Loading the key libraries

In [ ]:
import sys
import pandas as pd
import numpy as np


print(f"pandas version {pd.__version__}")
print(f"numpy version {np.__version__}")
print("\n".join(f"Python {sys.version}".split("|")))

## A comment on file formats

The most common simple format for tabular data is comma separated or tab separated (`csv` or `tsv`).

Newer formats such as `arrow` and `parquet` are more efficient in storage and faster to load.

Pandas 2.0 can handle these newer formats.


## Reading a dataframe


In [ ]:
# read from a csv file
penguins = pd.read_csv("data/penguins-raw.csv")
# read from a url
# url = "https://raw.githubusercontent.com/mcnakhaee/palmerpenguins/master/palmerpenguins/data/penguins-raw.csv"
# penguins = pd.read_csv(url)
# read from an excel file
# penguins = pd.read_excel('file.xlsx')
rows, cols = penguins.shape
print(f"Rows: {rows}, Columns: {cols}")
print(f"Columns:", "\n".join(penguins.columns))

## Series 

Each column of a dataframe is a series accessed by name.


In [ ]:
penguins["Culmen Length (mm)"]

Note the last row:
- Name
- Length
- dtype

Types are "inferred" by the read_csv function.

## Another example


In [ ]:
penguins['Date Egg']

Here the type is "object" which is the generic python object.  But these are clearly supposed to be dates. We'll fix that later.

## Alternative syntax


In [ ]:
# if the column name is simple, you can use a simpler syntax.
penguins.Sex

## Value Counts

The `value_counts` method returns a summary series.


In [ ]:
penguins['Island'].value_counts()

In [ ]:
penguins['Species'].value_counts()

## Selecting a subset of columns


In [ ]:
simpler = penguins[['Species', 'Body Mass (g)', 'Flipper Length (mm)']]
simpler.head()

## Index

A dataframe has an index, which can be just the numbers from 0 to N as in this case.


In [ ]:
penguins.index

## Columns and Rows

`loc` allows you to access individual elements.


In [ ]:
# The 23rd row
penguins.loc[23,:]

In [ ]:
penguins.loc[23,'Culmen Length (mm)']

In [ ]:
penguins.loc[23:28,['Sex','Date Egg']]

## Filtering

Filtering is done by using a boolean series as an index.  


In [ ]:
penguins['Sex']=='FEMALE'

In [ ]:
females = penguins[penguins['Sex']=='FEMALE']
females.head()

An alternative syntax is to use query.  The quoting rules here can be tricky.
The query is a string, and column names are set off by backticks.  Using
two different types of quotes allows the query to include a string.


In [ ]:
females = penguins.query("`Sex`=='FEMALE'")
females.head()

## Fancier filtering


In [ ]:
penguins[penguins["Flipper Length (mm)"]>penguins["Body Mass (g)"]/20]

## Missing values

Dealing with missing values is a central problem in data science.  One way to identify how many misssing values are out there is as follows:


In [ ]:
## Uses the fact that logical True counts as one, False as zero
## sum() method sums by columns
penguins.isna().sum()

Nearly all of the comments are empty.  What are they?


In [ ]:
comments = penguins['Comments'].value_counts()
comments

Let's save the comments separately and look at the rest.


In [ ]:
# drop normally drops rows, but with axis=1 it drops columns
penguins = penguins.drop('Comments',axis=1)

Various options:
- drop rows with missing values
- impute the missing values somehow

## Drop rows with missing values


In [ ]:
# This makes a boolean where a row is True provided at least one of its entries are NA
na_rows = (penguins.isna().any(axis=1))
print(f"{na_rows.sum()} rows have NA somewhere outside of comments")

In [ ]:
# here we keep rows only if no NA's.  Can also use notna().
penguins_nona = penguins.loc[~na_rows,:]

## Imputation

We saw above that culmen length has 2 missing values. We can use `fillna` to
replace the missing values with something (like the mean or median or zero).


In [ ]:
# using equality w/o copy creates another reference.
penguins_imputed = penguins.copy()
culmen_mean = penguins_imputed['Culmen Length (mm)'].mean() # how does this handle NA values?
print(f"Culmen length mean is {culmen_mean}")
penguins_imputed['Culmen Length (mm)'] = penguins_imputed['Culmen Length (mm)'].fillna(culmen_mean)

There are many other imputation methods.  For example, if the data is ordered, you can fill 
missing data with linear interpolation. (See the `interpolate` method).

## Data types

As we saw above, the types of the columns are inferred when the data is read.  But it's not always correct.  For example, the "Date Egg" column is supposed to be a date but it's shown as a generic python object. 

Using the correct type can greatly improve performance as generic Python arguments
are inefficient.

In pandas 1.0 strings are always treated as objects but in pandas 2.0 there is a `StringDtype`.

The most common types are:
- object
- float64
- datetime (datetime64[ns])
- int64
- bool 

One may also find categorical types. 


In [ ]:
penguins.dtypes

## Setting datatypes

Here's an example where we manually make sex a categorical type.


In [ ]:
penguins = penguins.astype({'Sex':'category'})
penguins['Sex']

One can also pass a dictionary setting the types of columns as an argument when
you read them from the csv file.

## Creating new columns 


Simplifying the species name.


In [ ]:
def first_word(x):
    return x.split()[0]
penguins['SimpleSpecies'] = penguins['Species'].map(first_word)

Rewriting the body mass in kilograms.


In [ ]:
penguins['Body Mass (kg)'] = penguins['Body Mass (g)']/1000

## Sorting


In [ ]:
penguins_small = penguins[['Species','Island','Body Mass (g)']]
penguins_small.sort_values('Body Mass (g)')
# ascending = False for descending order
# na_position = 'first' or 'last' (default is 'last')
# can also provide a key which is a function of prototype Series -> Series
# inplace = True doesn't return a new dataframe, sorts the given one in place

## Grouping

Grouping is a powerful tool.  Let's first
group our penguins by species.  The result is a "grouped" object which needs to pass through a summarize operation to
be useful. 


In [ ]:
penguins_by_species = penguins.groupby('Species')
penguins_by_species

## Summarizing


In [ ]:
# describe computes basic descriptive statistics
penguins_by_species['Body Mass (g)'].describe()

In [ ]:
# this fails because some columns aren't numeric
penguins_by_species.mean(numeric_only=True)

In [ ]:
penguins_by_species.count()

## MultiIndex


In [ ]:
penguins_by_sex_and_species = penguins.groupby(['Sex','Species'])
penguins_by_sex_and_species['Body Mass (g)'].describe().round()

In [ ]:
# pivot tables
penguins_by_sex_and_species['Body Mass (g)'].mean().reset_index().pivot(index='Sex',columns='Species',values='Body Mass (g)')

## Pandas plotting

Some simple plots are available directly from pandas.


In [ ]:
penguins[penguins['Species'].str.startswith("Adel")].groupby(['Sex'])['Body Mass (g)'].hist(bins=30,legend=True)